In [2]:
import backtrader as bt
import pandas as pd
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine
import talib as ta
import datetime


In [3]:
# indicators test
## import data
engine= create_engine('sqlite:///../dataset/us/us_ticker_price_yf.db')
df = pd.read_sql('AAPL', engine)
df.rename(columns={'hgih':'high'},inplace=True)
df['chg_pct'] = ((df.close - df.open)/df.open)*100
df['close'] = df['close'].apply(lambda x: round(x,3))
df = df.loc[9000:]
df.index = range(len(df))
df.to_csv('aapl_price.csv',index=None)

In [4]:
aapl_close = df['close'].values
aapl_macd = ta.MACD(aapl_close)

In [7]:
len_ =  len(df.index)
for i in range(50,len_):
    print("{}: {} {} {} ".format(df.datetime.loc[i], round(aapl_macd[0][i],2),round(aapl_macd[1][i],2),round(aapl_macd[2][i]),3))

2016-10-31: 0.19 0.37 0 
2016-11-01: 0.1 0.31 0 
2016-11-02: 0.03 0.26 0 
2016-11-03: -0.07 0.19 0 
2016-11-04: -0.16 0.12 0 
2016-11-07: -0.2 0.06 0 
2016-11-08: -0.21 0.0 0 
2016-11-09: -0.22 -0.04 0 
2016-11-10: -0.29 -0.09 0 
2016-11-11: -0.33 -0.14 0 
2016-11-14: -0.41 -0.19 0 
2016-11-15: -0.44 -0.24 0 
2016-11-16: -0.4 -0.28 0 
2016-11-17: -0.37 -0.29 0 
2016-11-18: -0.34 -0.3 0 
2016-11-21: -0.27 -0.3 0 
2016-11-22: -0.22 -0.28 0 
2016-11-23: -0.19 -0.26 0 
2016-11-25: -0.15 -0.24 0 
2016-11-28: -0.12 -0.22 0 
2016-11-29: -0.1 -0.19 0 
2016-11-30: -0.1 -0.17 0 
2016-12-01: -0.12 -0.16 0 
2016-12-02: -0.12 -0.15 0 
2016-12-05: -0.14 -0.15 0 
2016-12-06: -0.14 -0.15 0 
2016-12-07: -0.12 -0.14 0 
2016-12-08: -0.07 -0.13 0 
2016-12-09: -0.0 -0.1 0 
2016-12-12: 0.04 -0.07 0 
2016-12-13: 0.11 -0.04 0 
2016-12-14: 0.17 0.0 0 
2016-12-15: 0.22 0.05 0 
2016-12-16: 0.26 0.09 0 
2016-12-19: 0.31 0.13 0 
2016-12-20: 0.34 0.18 0 
2016-12-21: 0.37 0.21 0 
2016-12-22: 0.37 0.25 0 
2016-12-23:

In [14]:
df.dtypes

open         float64
high         float64
low          float64
close        float64
Adj Close    float64
volume         int64
datetime      object
chg_pct      float64
dtype: object

In [33]:
# indicators test
## import data
engine= create_engine('sqlite:///../dataset/us/us_ticker_price_yf.db')
df = pd.read_sql('AAPL', engine)
df.rename(columns={'hgih':'high'},inplace=True)
df['chg_pct'] = ((df.close - df.open)/df.open)*100
df['close'] = df['close'].apply(lambda x: round(x,3))
df = df.loc[9000:]
df.index = range(len(df))
df.datetime = df.datetime.apply(lambda x: x[0:4] + x[5:7] + x[8:10])

In [36]:
df.datetime = df.datetime.astype(int)
df.dtypes
df.to_csv('aapl_price.csv',index=None)

In [38]:
apple_df = pd.read_csv('aapl_price.csv')
apple_df.dtypes

open         float64
high         float64
low          float64
close        float64
Adj Close    float64
volume         int64
datetime       int64
chg_pct      float64
dtype: object

In [41]:
class trading_strategy(bt.Strategy):
    # 全局设定交易策略的参数
    params=(
        ('time_period',200), 
    )

    # 初始化变量
    def __init__(self):
        # 指定价格序列
        self.dataclose = self.datas[0].close
        self.ourder = None
        self.buyprice = None
        self.buycomm = None

        # 添加移动平均线指标，内置了talib模块
        self.ema = bt.indicators.ExponentialMovingAverage(
            self.datas[0], period = self.params.time_period
        )
        # macd快线
        self.macd_line = bt.indicators.MACD(self.dataclose)[0]
        # macd慢线
        self.signal_line = bt.indicators.MACD(self.dataclose)[1]
    
    # 设置交易
    def next(self):
        # 获取收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 检查是否正在下单，如果不是的话不能提交第二个订单，保证一个交易对只有一个订单
        if self.order:
            return
        
        # 检查是否已经买入
        if not self.position:
            # 
            if self.macd_line[0] > self.signal_line[0]:
                self.order = self.buy()
        else:
            if self.macd_line[0] < self.signal_line[0]:
                self.order = self.sell()

    # 设置交易记录
    def log(self, txt, df=None):
        ''' logging function for this strategy'''
        dt = dt or self.dates[0].datetime.date[0]
        print('%s, %s' % (dt.isoformat(), txt))
    
    @staticmethod
    def percent(today, yesterday):
        return float(today - yesterday) / today

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return 
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f'%
                    (order.executed.price, 
                     order.executed.value, 
                     order.executed.comm)
                )
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                self.bar_executed_close = self.dataclose[0]
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


In [42]:
# 设置交易的策略
if __name__ == '__main__':
    # 初始化模型
    cerebro = bt.Cerebro()

    # 添加交易策略
    cerebro.addstrategy(trading_strategy)

    # 加载数据
    data = bt.feeds.GenericCSVData(
        dataname = 'aapl_price.csv',
        fromdate = datetime.datetime(2016,10,11), 
        todate = datetime.datetime(2017,12,4),
        # 定义自己的数据格式
        dtformat = '%Y%m%d', 
        datetime = 6, 
        open = 0, 
        high = 1, 
        low = 2, 
        close = 3, 
        volume = 5, 
    )

    cerebro.adddata(data)

    # 设置初始资金
    dmoney = 100000.0
    cerebro.broker.setcash(dmoney)
    dcash0 = cerebro.broker.startingcash
    
    # 设置佣金
    cerebro.broker.setcommission(commission=0.005)
    # 设置每次交易购买的股数
    cerebro.addsizer(bt.sizers.FixedSize,stake=10)

    # 执行策略前的资金
    print('Starting Portfilo Value: %.2f' % cerebro.broker.getvalue())
    # 运行策略
    cerebro.run()
    # 执行策略后的资金
    print('Final Portfilo Value: %.2f' % cerebro.broker.getvalue())

    # 可视化
    cerebro.plot()


Starting Portfilo Value: 100000.00


IndexError: array index out of range